In [ ]:
# Code resource: https://github.com/Gananath/DrugAI

'''
original Author: Gananath R
DrugAI-WGAN: 
Contact: https://github.com/gananath
'''

In [3]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv1D, UpSampling1D, MaxPooling1D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint
import keras.backend as K

Using TensorFlow backend.


# Function Area

In [4]:
def wasserstein_loss(y_true, y_pred):
    '''Loss dunction'''
    return K.mean(y_true * y_pred)
    

def dimX(x, ts):
    '''time step addtition to feature'''
    x = np.asarray(x)
    newX = []
    for i, c in enumerate(x):
        newX.append([])
        for j in range(ts):
            newX[i].append(c)
    return np.array(newX)


def dimY(Y, ts, char_idx, chars):
    '''time step addtition to feature'''
    temp = np.zeros((len(Y), ts, len(chars)), dtype=np.bool)
    for i, c in enumerate(Y):
        for j, s in enumerate(c):
            # print i, j, s
            temp[i, j, char_idx[s]] = 1
    return np.array(temp)


def train_test_split(X, y, percentage=0.75):
    p = int(len(X) * percentage)
    X_train = X[0:p]
    Y_train = y[0:p]
    X_test = X[p:]
    Y_test = y[p:]
    return X_train, X_test, Y_train, Y_test


def prediction(preds):
    '''prediction of argmax'''
    y_pred = []
    for i, c in enumerate(preds):
        y_pred.append([])
        for j in c:
            y_pred[i].append(np.argmax(j))
    return np.array(y_pred)


def seq_txt(y_pred, idx_char):
    '''sequence to text conversion'''
    newY = []
    for i, c in enumerate(y_pred):
        newY.append([])
        for j in c:
            newY[i].append(idx_char[j])

    return np.array(newY)


def smiles_output(s):
    '''joined smiles output'''
    smiles = np.array([])
    for i in s:
        j = ''.join(str(k) for k in i)
        smiles = np.append(smiles, j)
    return smiles

In [5]:
np.random.seed(2017)

# Read Data

In [6]:
# Read csv file
data = pd.read_csv('stahl.csv')
data = data.reindex(np.random.permutation(data.index))
print('Number of data:',len(data))
data.head(10)

Number of data: 335


,SMILES,cox2,estrogen,gelatinase,neuramidase,kinase,thrombin,none
21,N[S](=O)(=O)C1=CC=C(C=C1)C2=CC(=N[N]2C3=CC=C(F...,1,0,0,0,0,0,0
30,C[S](=O)(=O)C1=CC=C(C=C1)C2=C(C=C(F)C(=C2)F)C3...,1,0,0,0,0,0,0
102,C[S](=O)(=O)C1=CC=C(C=C1)C2=C(C(=C)C2)C3=CC=CC=C3,1,0,0,0,0,0,0
250,CCC(CC)NC1=NC(=CC=N1)C2=C(N=C[N]2C3CC[NH](C)CC...,0,0,0,0,1,0,0
299,CCOC1=C(Cl)C2=C(C=C(C=C2)[NH+]=C(N)N)C(=O)O1,0,0,0,0,0,1,0
172,CC1=C(C(OC2=C1C=C(O)C=C2)C3=CC=C(OCC[NH]4CCCCC...,0,1,0,0,0,0,0
214,CNC(=O)C(CC1=CC=CC=C1)NC(=O)C(CC(C)C)NC(CCN2C(...,0,0,1,0,0,0,0
145,CC12CCC3C(CCC4=C3C=CC(=C4)O)C1CCCC2O,0,1,0,0,0,0,0
50,COC1=CC(=CC=C1)SCC2=N[N](C3=CC=C(C=C3)[S](C)(=...,1,0,0,0,0,0,0
121,CC(C1=CC(=C(O)C(=C1)C(C)(C)C)C(C)(C)C)=C2NC(=N...,1,0,0,0,0,0,0


In [7]:
# Take the SMILES string
Y=data.SMILES
print(type(Y))
Y[0:5]

<class 'pandas.core.series.Series'>


21     N[S](=O)(=O)C1=CC=C(C=C1)C2=CC(=N[N]2C3=CC=C(F...
30     C[S](=O)(=O)C1=CC=C(C=C1)C2=C(C=C(F)C(=C2)F)C3...
102    C[S](=O)(=O)C1=CC=C(C=C1)C2=C(C(=C)C2)C3=CC=CC=C3
250    CCC(CC)NC1=NC(=CC=N1)C2=C(N=C[N]2C3CC[NH](C)CC...
299         CCOC1=C(Cl)C2=C(C=C(C=C2)[NH+]=C(N)N)C(=O)O1
Name: SMILES, dtype: object

In [8]:
X = data.iloc[:,1:7] # in this case, take the data of column 1:7
# DataFrame.iloc: Purely integer-location based indexing for selection by position.
X = X.values
X = X.astype('int')
type(X)

numpy.ndarray

# Data Preprocessing

In [9]:
# padding smiles to same length by adding "|" at the end of smiles
maxY = Y.str.len().max() + 11 # find the max length of SMILES
y = Y.str.ljust(maxY, fillchar='|') # the maxY-th number frim left side, fill '|'
ts = y.str.len().max()
print ("ts={0}".format(ts))

# CharToIndex and IndexToChar functions
chars = sorted(list(set("".join(y.values.flatten()))))
print('total chars:', len(chars))

char_idx = dict((c, i) for i, c in enumerate(chars))
idx_char = dict((i, c) for i, c in enumerate(chars))

y_dash = dimY(y, ts, char_idx, chars)
x_dash = X
print("Shape\n X={0} Y={1}".format(x_dash.shape, y_dash.shape))

ts=116
total chars: 25
Shape
 X=(335, 6) Y=(335, 116, 25)


# Building Model

In [11]:
def Discriminator(y_dash, dropout=0.4, lr=0.00001, PATH="Dis.h5"):
    """Creates a discriminator CNN model that takes a sequence as input and outputs a single value(Real or Fake) """
    model = Sequential()
    # First convolution layer
    model.add(Conv1D(input_shape=(y_dash.shape[1], y_dash.shape[2]),
                     nb_filter=25,
                     filter_length=4,
                     border_mode='same'))
    model.add(LeakyReLU())
    model.add(Dropout(dropout))
    model.add(MaxPooling1D())
    
    # 2-th convolution layere
    model.add(Conv1D(nb_filter=10,
                     filter_length=4,
                     border_mode='same'))
    model.add(LeakyReLU())
    model.add(Dropout(dropout))
    model.add(MaxPooling1D())
    model.add(Flatten())
    
    # 3-th layer DNN
    model.add(Dense(64))
    model.add(LeakyReLU())
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.add(Activation('linear'))

    opt = Adam(lr, beta_1=0.5, beta_2=0.9)

    #reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.9, patience=30, min_lr=0.000001, verbose=1)
    checkpoint_D = ModelCheckpoint(filepath=PATH, 
                                   verbose=1, 
                                   save_best_only=True)

    model.compile(optimizer=opt,
                  loss=wasserstein_loss,
                  metrics=['accuracy'])
    return model, checkpoint_D



def Generator(x_dash, y_dash, dropout=0.4, lr=0.00001, PATH="Gen.h5"):
    """Creates a Generator DNN model that takes OneHot as input and outputs a sequence """
    model = Sequential()
    
     # Input layer
    model.add(Dense(x_dash.shape[1], activation="relu", input_shape=(6,)))
    model.add(Dense( int(y_dash.shape[1]/4 * y_dash.shape[2]*8), #(None, 5800)
             activation="relu"))
    model.add(Reshape((int(y_dash.shape[1] / 4), y_dash.shape[2] * 8)))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dropout(dropout))
    model.add(UpSampling1D())
    
    # 1-th convolution layere
    model.add(Conv1D(y_dash.shape[2] * 8, kernel_size=4, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dropout(dropout))
    model.add(UpSampling1D())
    
    # 2-th convolution layere
    model.add(Conv1D(y_dash.shape[2] * 2, kernel_size=4, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dropout(dropout))
    
    # 3-th convolution layere
    model.add(Conv1D(y_dash.shape[2] * 1, kernel_size=4, padding="same"))
    model.add(Activation("softmax"))
    opt = Adam(lr, beta_1=0.5, beta_2=0.9)
    checkpoint_G = ModelCheckpoint(filepath=PATH, 
                                   verbose=1, 
                                   save_best_only=True)
    model.compile(loss="categorical_crossentropy",
                  optimizer=opt,
                  metrics=['accuracy'])
    return model, checkpoint_G


def Gan(PATH="GAN.h5", lr=0.00001):
    '''GAN model'''
    GAN = Sequential()
    GAN.add(G) # Generating mosel
    
    D.trainable = False # Weight of dis model will not change when training GAN
    # How model.trainable = False works in keras (GAN model)
    # https://gist.github.com/naotokui/a9274f12af9d946e99b6df73a5d2af6d
    GAN.add(D) # Discriminator model
    checkpoint_GAN = ModelCheckpoint(filepath=PATH, 
                                     verbose=1, 
                                     save_best_only=True)
    GAN.compile(loss=wasserstein_loss,
                optimizer=Adam( lr,
                                beta_1=0.5,
                                beta_2=0.9))
    return GAN, checkpoint_GAN

In [12]:
BATCH_SIZE = 32
HALF_BATCH = int(BATCH_SIZE / 2)
CLIP = 0.01
epochs = 100000
n_critic = 5

In [ ]:
# Building Model
G, checkG = Generator(x_dash, y_dash)
D, checkD = Discriminator(y_dash)
GAN, checkGAN = Gan()
# Enable training in discrimator
D.trainable = True

# Training Model

In [14]:
for epoch in range(epochs):
    for _ in range(n_critic):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half batch of sequence
        idx = np.random.randint(0, y_dash.shape[0], HALF_BATCH)
        imgs = y_dash[idx]

        # noise = np.random.normal(0, 1, (HALF_BATCH, 100))
        noise = x_dash[0:HALF_BATCH]
        # Generate a half batch of new sequence
        gen_imgs = G.predict(noise)

        # Train the discriminator
        d_loss_real = D.train_on_batch(imgs, -np.ones((HALF_BATCH, 1)))  # linear activation
        d_loss_fake = D.train_on_batch(gen_imgs, np.ones((HALF_BATCH, 1)))
        d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

        # Clip discriminator weights
        for l in D.layers:
            weights = l.get_weights()
            weights = [np.clip(w, -CLIP, CLIP) for w in weights]
            l.set_weights(weights)

            
        # ------------------------
        #  Train Generator
        # ------------------------
        # noise = np.random.normal(0, 1, (BATCH_SIZE, 100))
        noise = x_dash[0:BATCH_SIZE]
        # Train the generator
        g_loss = GAN.train_on_batch(
            noise, -np.ones((BATCH_SIZE, 1)))  # linear activation
    print("%d [D loss: %f] [G loss: %f]" % (epoch, 1 - d_loss[0], 1 - g_loss))
    
    
    if epoch % (epochs / 1000) == 0:
        # for saving files
        G.save_weights(".\\Output\\Gen.h5")
        D.save_weights(".\\Output\\Dis.h5")
        GAN.save_weights(".\\Output\\GAN.h5")

        # For Prediction
        Ghash, checkG = Generator(x_dash, y_dash)
        print("Prediction")
        Ghash.load_weights('.\\output\\Gen.h5')
        x_pred = [[0, 0, 0, 1, 0, 0],
                  [0, 1, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 1]]
        preds = Ghash.predict(x_pred)
        y_pred = prediction(preds)
        y_pred = seq_txt(y_pred, idx_char)
        s = smiles_output(y_pred)
        print(s)
        # end prediction'''

C:\Users\Eric\Anaconda3\envs\Tensorflow\lib\site-packages\keras\engine\training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.999995] [G loss: 1.000009]
Prediction
[ ']#FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFl'
 ']#FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFl'
 ']#FFFFFFFFFFFFFFFFFFFFFFFFFF##FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF#FFFFFFFFl']
1 [D loss: 0.999992] [G loss: 1.000012]
2 [D loss: 0.999995] [G loss: 1.000011]
3 [D loss: 0.999992] [G loss: 1.000013]
4 [D loss: 0.999991] [G loss: 1.000014]
5 [D loss: 0.999995] [G loss: 1.000014]
6 [D loss: 0.999997] [G loss: 1.000016]
7 [D loss: 0.999997] [G loss: 1.000017]
8 [D loss: 0.999994] [G loss: 1.000018]
9 [D loss: 1.000007] [G loss: 1.000020]
10 [D loss: 0.999997] [G loss: 1.000017]
11 [D loss: 1.000007] [G loss: 1.000019]
12 [D loss: 1.000004] [G loss: 1.000017]
13 [D loss: 1.000010] [G loss: 1.000016]


KeyboardInterrupt: 